In [1]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc
import sqlite3
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import scipy.stats as stats
import plotly.express as px

_conn = sqlite3.connect("airbnb_cartagena.sqlite")
df_attr = pd.read_sql_query("SELECT * FROM Attributes", _conn, dtype={"ID": str})
df_ts = pd.read_sql_query("SELECT * FROM TimeSeriesRaw", _conn, dtype={"ID": str})
df_ts_interp = pd.read_sql_query("SELECT * FROM TimeSeriesInterpolated", _conn, dtype={"ID": str})
_conn.close()

In [2]:
red = "#7e0d24"  # dark red color for plots

In [12]:
import re
dates = [col for col in df_ts.columns if re.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}", col)]

In [25]:
import json
import pandas as pd
import plotly.express as px

_conn = sqlite3.connect("airbnb_cartagena.sqlite")
df_attr = pd.read_sql_query("SELECT * FROM Attributes", _conn, dtype={"ID": str})
df_ts = pd.read_sql_query("SELECT * FROM TimeSeriesRaw", _conn, dtype={"ID": str})
df_ts_interp = pd.read_sql_query("SELECT * FROM TimeSeriesInterpolated", _conn, dtype={"ID": str})
_conn.close()

with open("assets/cartagena.geojson") as f:
    cartagena_geo = json.load(f)


df_region = (
    df_attr
    .groupby("barrio")["Base fee"]
    .mean()
    .reset_index(name="avg_base_fee")
)

fig = px.choropleth_mapbox(
    df_region,
    geojson=cartagena_geo,
    locations="barrio",                   # matches df_region.barrio -> geojson.properties.name
    color="avg_base_fee",
    featureidkey="properties.name",       # path in geojson to match ‘locations’
    mapbox_style="carto-darkmatter",      # or "open-street-map", etc.
    center={"lat": 10.4, "lon": -75.5},    # approximate Cartagena center
    zoom=11,
    opacity=0.7,
    color_continuous_scale="Viridis",
    labels={"avg_base_fee": "Avg. Base Fee (USD)"},
    title="Average Nightly Base Fee by Neighborhood"
)

fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

fig


KeyError: 'barrio'

In [40]:
import plotly.express as px


def map_base_fee_scatter():
    """
    Point-map of each listing colored and sized by Base fee.
    """
    fig = px.scatter_map(
        df_attr,
        lat="latitude",
        lon="longitude",
        color="Base fee",
        size="Base fee",
        hover_name="Name",
        hover_data=["Property type", "Cleaning fee", "Person capacity"],
        color_continuous_scale="Viridis",
        size_max=12,
        zoom=11,
        center={"lat": df_attr["latitude"].mean(), "lon": df_attr["longitude"].mean()},
        title="Cartagena Listings: Base Fee Distribution",
    )
    fig.update_layout(mapbox_style="carto-darkmatter", margin={"r":0,"t":40,"l":0,"b":0})
    return fig

map_base_fee_scatter()

In [58]:
import plotly.express as px

def map_base_fee_density():
    """
    Density-map of listing locations, weighted by Base fee.
    """
    fig = px.density_map(
        df_attr,
        lat="latitude",
        lon="longitude",
        z="Base fee",
        radius=20,
        hover_name="Name",
        hover_data=["Base fee", "Property type"],
        color_continuous_scale="Hot_r",
        map_style="carto-darkmatter",
        zoom=11,
        title="Cartagena Base Fee Density"
    )
    fig.update_layout(
        margin={"r":0,"t":40,"l":0,"b":0},
        paper_bgcolor="#111111",   
        plot_bgcolor="#111111",   
        font=dict(color="white") 
    )
    return fig

map_base_fee_density()